# Context Aware Image Inpainting using DCGANs


## Objectives

1. Generate Samples from a High Dimensional Distribution using DCGANs 
2. Show the robustness of our model by making sure it doesn’t memorize the training set
3. Perform Image Completion using the above model to fill in the incomplete parts of our images


## Why study generative modelling?

One might legitimately wonder why generative models are worth studying, especially generative models that are only capable of generating data rather than providing an estimate of the density function. After all, when applied to images, such models seem to merely provide more images, and the world has no shortage of images. 
There are several reasons to study generative models, including: 

* Training and sampling from generative models is an excellent test of our ability to represent and manipulate high-dimensional probability distributions. High-dimensional probability distributions are important objects in a wide variety of applied math and engineering domains. 
* 	Generative models can be incorporated into reinforcement learning in several ways. 
*	Generative models can be trained with missing data and can provide predictions on inputs that are missing data. 

One particularly interesting case of missing data is semi-supervised learning, in which the labels for many or even most training examples are missing. Modern deep learning algorithms typically require extremely many labeled examples to be able to generalize well. Semi-supervised learning is one strategy for reducing the number of labels. The learning algorithm can improve its generalization by studying a large number of unlabeled examples which, which are usually easier to obtain. Generative models, and GANs in particular, are able to perform semi-supervised learning reasonably well. 


## The GAN Framework

The basic idea of GANs is to set up a game between two players. One of them is called the *generator*. The generator creates samples that are intended to come from the same distribution as the training data. The other player is the *discriminator*. The discriminator examines samples to determine whether they are real or fake. The discriminator learns using traditional supervised learning techniques, dividing inputs into two classes (real or fake). The generator is trained to fool the discriminator. We can think of the generator as being like a counterfeiter, trying to make fake money, and the discriminator as being like police, trying to allow legitimate money and catch counterfeit money. To succeed in this game, the counterfeiter must learn to make money that is indistinguishable from genuine money, and the generator network must learn to create samples that are drawn from the same distribution as the training data. 


## Training Process

The training process consists of simultaneous $SGD$. On each step, two minibatches are sampled: a minibatch of x values from the dataset and a minibatch of z values drawn from the model’s prior over latent variables. Then two gradient steps are made simultaneously: one updating $\theta{(D)}$ to reduce $J(D)$ and one updating $\theta{(G)}$ to reduce $J(G)$. In both cases, it is possible to use the gradient-based optimization algorithm of your choice. Adam (Kingma and Ba, 2014) is usually a good choice. Many authors recommend running more steps of one player than the other, but as of late 2016, the author’s opinion is that the protocol that works the best in practice is simultaneous gradient descent, with one step for each player. 


1. $G$ is a simple neural net (e.g. just one fully connected hidden layer). It takes a vector as input (e.g. 100 dimensions) and produces an image as output.
2.	$D$ is a simple neural net (e.g. just one fully connected hidden layer). It takes an image as input and produces a quality rating as output (0-1, so sigmoid).
3.	You need a training set of things to be generated, e.g. images of human faces.
4.	Let the batch size be B.
5.	$G$ is trained the following way:
    + Create B vectors of 100 random values each, e.g. sampled uniformly from $[-1, +1]$. (Number of values per components depends on the chosen input size of G.)
    +	Feed forward the vectors through $G$ to create new images.
    +	Feed forward the images through $D$ to create ratings.
    +	Use a cross entropy loss on these ratings. All of these (fake) images should be viewed as $label = 0$ by $D$. If $D$ gives them $label=1$, the error will be low ($G$ did a good job).
    +	Perform a backward pass of the errors through $D$ (without training $D$). That generates gradients/errors per image and pixel.
    +	Perform a backward pass of these errors through $G$ to train $G$.
6.	$D$ is trained the following way:
    +	Create B/2 images using $G$ (again, B/2 random vectors, feed forward through G).
    +	Chose B/2 images from the training set. Real images get $label=1$.
    +	Merge the fake and real images to one batch. Fake images get $label=0$.
    +	Feed forward the batch through $D$.
    +	Measure the error using cross entropy.
    +	Perform a backward pass with the error through $D$.
7.	Train $G$ for one batch, then $D$ for one (or more) batches. Sometimes $D$ can be too slow to catch up with $D$, then you need more iterations of $D$ per batch of $G$.



## Cost functions 

The cost used for the discriminator is:

\begin{align*}
J^{(D)}(\theta^{(D)},\theta^{(G)} = -\frac{1}{2}E_{x \in p_{data}}logD(x) - \frac{1}{2}E_z log(1 - D(G(z)))
\end{align*}

This is just the standard cross-entropy cost that is minimized when training a standard binary classifier with a sigmoid output. The only difference is that the classifier is trained on two minibatches of data; one coming from the dataset, where the label is $1$ for all examples, and one coming from the generator, where the label is $0$ for all examples. We see that by training the discriminator, we are able to obtain an estimate of the ratio $\frac{pdata(x)}{ pmodel(x)}$ at every point $x$. Estimating this ratio enables us to compute a wide variety of divergences and their gradients. This is the key approximation technique that sets GANs apart from variational autoencoders and Boltzmann machines. The GAN approximation is subject to the failures of supervised learning: *overfitting and underfitting*. In principle, with perfect optimization and enough training data, these failures can be overcome. Other models make other approximations that have other failures. 

Because the GAN framework can naturally be analyzed with the tools of game theory, we call GANs “adversarial.” But we can also think of them as cooperative, in the sense that the discriminator estimates this ratio of densities and then freely shares this information with the generator. From this point of view, the discriminator is more like a teacher instructing the generator in how to improve than an adversary. So far, this cooperative view has not led to any particular change in the development of the mathematics. 

### Minimax Game


We train D to maximize the probability of assigning the correct label to both training examples and samples from G.


We simultaneously train G to minimize $log(1 − D(G(z)))$. In other words, D and G play the following two-player minimax game with value function V (G, D) :

\begin{align*}
min_{G}max_{D}V(D,G) = E_{x \in p_{data}(x)} logD(x) + E_{z \in p_{z}(z)}log(1 - D(G(z)) 
\end{align*}

Optimizing $D$ to completion in the inner loop of training is computationally prohibitive, and on finite datasets would result in overfitting. The training criterion allows one to recover the data generating distribution as $G$ and $D$ are given enough capacity, i.e., in the non-parametric limit. Instead, we alternate between $k$ steps of optimizing $D$ and one step of optimizing $G$. This results in D being maintained near its optimal solution, so long as $G$ changes slowly enough. 




## The DCGAN architecture 

Most GANs today are at based on the DCGAN architecture (Radford et al., 2015). DCGAN stands for “deep, convolution GAN.” Though GANs were both deep and convolutional prior to DCGANs, the name DCGAN is useful to refer to this specific style of architecture. Some of the key insights of the DCGAN architecture were to: 
+	Replace any pooling layers with *strided convolutions* (discriminator) and *fractional-strided convolutions* (generator). 
+	Use *batch normalization* layers in most layers of both the discriminator and the generator, with the two minibatches for the discriminator normalized separately. The last layer of the generator and first layer of the discriminator are not batch normalized, so that the model can learn the correct mean and scale of the data distribution. 
+	Remove fully connected hidden layers for deeper architectures. 
+	Use *ReLU* activation in generator for all layers except for the output, which uses Tanh. 
+	Use *LeakyReLU* activation in the discriminator for all layers. 
+	The use of the *Adam optimizer* rather than SGD with momentum. 




# Improvements

## One Sided Label Smoothing

GANs are intended to work when the discriminator estimates a ratio of two densities, but deep neural nets are prone to producing highly confident outputs that identify the correct class but with too extreme of a probability. This is especially the case when the input to the deep network is adversarially constructed the classifier tends to linearly extrapolate and produce extremely confident predictions. 

To encourage the discriminator to estimate soft probabilities rather than to extrapolate to extremely confident classification, we can use a technique called *one-sided label smoothing .*

The idea of one-sided label smoothing is to replace the target for the real examples with a value slightly less than one, such as $.9$. This prevents extreme extrapolation behavior in the discriminator; if it learns to predict extremely large logits corresponding to a probability approaching $1$ for some input, it will be penalized and encouraged to bring the logits back down to a smaller value.  It is important to not smooth the labels for the fake samples. Suppose we use a target of $1− \alpha$ for the real data and a target of $0+\beta$ for the fake samples. Then the optimal discriminator function is 


\begin{align}
    D^*(x) = \frac{(1 − \alpha)p_{data}(x) + \beta p_{model}(x)}{p_{data}(x) + p_{model}(x)}$
\end{align}

When $\beta$ is zero, then smoothing by $\alpha$ does nothing but scale down the optimal value of the discriminator. When $\beta$ is nonzero, the shape of the optimal discriminator function changes. In particular, in a region where $p_{data}(x)$ is very small and $p_{model}(x)$ is larger, $D^*(x)$ will have a peak near the spurious mode of $p_{model}(x)$. The discriminator will thus reinforce incorrect behavior in the generator; the generator will be trained either to produce samples that resemble the data or to produce samples that resemble the samples it already makes. 
Label smoothing is an excellent regularizer in the context of convolutional networks for object recognition. One reason that label smoothing works so well as a regularizer is that it does not ever encourage the model to choose an incorrect class on the training set, but only to reduce the confidence in the correct class. Other regularizers such as weight decay often encourage some misclassification if the coefficient on the regularizer is set high enough. *Warde-Farley and Goodfellow (2016)* showed that label smoothing can help to reduce vulnerability to adversarial examples, which suggests that label smoothing should help the discriminator more efficiently learn to resist attack by the generator. 


## Minibatch Discrimination

One of the main failure modes for GAN is for the generator to collapse to a parameter setting where it always emits the same point. When collapse to a single mode is imminent, the gradient of the discriminator may point in similar directions for many similar points. Because the discriminator processes each example independently, there is no coordination between its gradients, and thus no mechanism to tell the outputs of the generator to become more dissimilar to each other. Instead, all outputs race toward a single point that the discriminator currently believes is highly realistic. After collapse has occurred, the discriminator learns that this single point comes from the generator, but gradient descent is unable to separate the identical outputs. The gradients of the discriminator then push the single point produced by the generator around space forever, and the algorithm cannot converge to a distribution with the correct amount of entropy. An obvious strategy to avoid this type of failure is to allow the discriminator to look at multiple data examples in combination, and perform what we call minibatch discrimination. The concept of minibatch discrimination is quite general: any discriminator model that looks at multiple examples in combination, rather than in isolation, could potentially help avoid collapse of the generator. 


### Process

Let $f(x_i) \in R^A$ denote a vector of features for input $x_i$, produced by some intermediate layer in the discriminator. We then multiply the vector $f(x_i)$ by a tensor $T \in R^A \times R^B x R^C$, which results in a matrix $M_i \in R^BxR^C$. We then compute the $L1$-distance between the rows of the resulting matrix $M_i$ across samples $i \in {1,2,...,n}$
and apply a negative exponential $c_b (x_i , x_j ) = exp(−|| M_{i,b}− M_{j,b} || _{L1})$. The output $o(x_i)$ for this minibatch layer for a sample $x_i$ is then defined as the sum of the $c_b(x_i,x_j)$’s to all other samples:

\begin{align}
o(x_i)b =  \sum_{j=1}^n c_b(x_i,x_j) \in \mathbb{R} \\
o(x_i)= [o(x_i)_1 ,o(x_i)_2 ,...,o(x_i)_B]  \in \mathbb{R}^B \\
o(X) \in \mathbb{R}^{n \times B} \\
\end{align}

*In other words*,

+ Allow $D$ to look at multiple images in the same minibatch.
+ Feed the features (of each image) extracted by an intermediate layer of $D$ through a linear operation, resulting in a matrix per image.
+ Compute the $L1$ distances between these matrices.
+ Let $D$ make its judgement (fake/real image) based on the features extracted from the image and these distances.
+ Add this mechanism so that the diversity of images generated by $G$ increases (which should also prevent collapses).

## Instance Noise

+ A major reason for GANs' instability may be that the generative distributions are weird, degenerate, and their support don't generally overlap with the true data distribution.
+ This makes the nice theory break down and may lead to unstable behavior.
+ Adding noise to both real and synthetic data during training might help overcome these problems



In an idealised GAN algorithm, each iteration consisting of the following steps 

1. We train the discriminator $D$ via logistic regression between our generative model $q_\theta$ vs true data $p$ until convergence
2. We extract from $D$ an estimate of the logarithmic likelihood ratio $s(y)=\frac{logq_\theta(y)}{p(y)}$
3. We update \theta by taking a stochastic gradient step with objective function $E_y \in q_\theta(y)$

But there are problems in convergence due to the following assumptions,

1. that the log-likelihood-ratio $s(y)=\frac{logq_\theta(y)}{p(y)}$ is finite, or 
2. that the Jensen-Shannon divergence $JS[q_\theta‖p]$ is a well-behaved function of \theta and 
3. that the Bayes-optimal solution to the logistic regression problem is unique: there is a single optimal discriminator that does a much better job than any other classifier.

But any of the following might not be true especially that there may be a large set of near-optimal discriminators whose loss is very close to the Bayes optimum. Thus, for a fixed q_\theta and p, training the discriminator $D$
might lead to a different near-optimal solution each time depending on initialisation. And, each of these near-optimal solutions might provide very different gradients (or no useful gradients at all) to the generator.


We add gaussian additive noise to the layer of the discriminator. We add noise to samples from both $q_\theta$ and $p$
The noisy distributions now overlap, the log-likelihood-ratio is well-behaved, and the JS divergence between the two noisy distributions is a non-constant function of $\theta$.

## Image Completion 


We consider using both D and G for reconstruction. To quantify the “closest” mapping from $y$ to the reconstruction, we define a function consisting of contextual loss $L_{contexual}$ and $L_{perceptual}$.



### Contexual Loss

We need to incorporate the information from the uncorrupted portion of the given image. The contextual loss is used to measure the context similarity between the reconstructed image and the uncorrupted portion, which is defined as 

\begin{align*}
L_{contexual}(z) = ||M \odot G(z) - M \odot y||_1
\end{align*}

where $M$ denotes the binary mask of the uncorruption and $\odot$ denotes the element-wise product operation. The corrupted portion is not used in the loss. The choice of $L1$ norm is empirical. 


### Perceptual Loss

The perceptual loss encourages the reconstructed image to be similar to the samples drawn from the training set. 
This is achieved by updating $z$ to fool $D$. As a result, $D$ will predict $G(z)$ to be from the data with a high probability. We use the same loss for fooling $D$ as in GAN:

\begin{align*}
L_{perceptual}(z) = log(1 - D(G(z))
\end{align*}

Without $L_{perceptual}$, the mapping from $x$ to $z$ can converge to a perceptually implausible result. Some reconstructed images tend to be unrealistic. 




### Image Reconstruction

With the defined perceptual and contextual losses, the corrupted image can be mapped to the closest $z$ in the latent representation space. $z$ is updated using back-propagation with the total loss: 

\begin{align*}
\hat{z} = argmin_{z}L_{contexual}(z) + \lambda L_{perceptual}(z)
\end{align*}

where $\lambda$ is a weighting parameter. In practice, $\lambda$ has to be relatively small to constrain the recovered image with the input pixels. After finding $\hat{z}$, the inpainting can be obtained by: 

\begin{align*}
x_{reconstructed} = M \odot y + (1 - M) \odot G{(\hat{z})}
\end{align*}